In [1]:
from dao.mongodbdao import MongoDBDao
from tfidf.TF_IDF import TF_IDF
import numpy as np 

dao = MongoDBDao({
    "host": "127.0.0.1",
    "db": "default",
    "wordindex_collection": "wordindex",
    "pagedetails_collection": "pagedetails",
    "username": "root",
    "password": "spezanw"
})

# Speicherbedarf

In [2]:
tfidf = TF_IDF(dao)

TF-IDF module: initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.276386.php'} contains no words
TF-IDF module: initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.461948.php'} contains no words
TF-IDF module: initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.587702.php'} contains no words
TF-IDF module: initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.85381.php'} contains no words


In [3]:
def measureTF_IDF_Index():
    n_urls = len(tfidf.doc_term_tf_idfs)
    #url_lengths = np.sum([len(url) for url in self.doc_term_tf_idfs])
    pass

n_urls = len(tfidf.doc_term_tf_idfs)
urls_length = np.sum([len(url) for url in tfidf.doc_term_tf_idfs]) #String

n_terms = 0
terms_length = 0 #String
n_tf_idf_values = 0 #np.Float64
for docid, term_tf_idfs in tfidf.doc_term_tf_idfs.items():
    for term, tf_idf in term_tf_idfs.items():
        terms_length += len(term)
        n_terms += 1
        n_tf_idf_values += 1

strings_memory = urls_length + n_urls*40\
                    + n_urls*64\
                    + n_urls*240\
                    + terms_length + n_terms*40
tf_idf_values_memory = n_tf_idf_values*32
print("I calculated:\t\t", strings_memory+tf_idf_values_memory)

import sys
print("python calculated:\t", sys.getsizeof(tfidf.doc_term_tf_idfs))

manual_size = 0
for docid, term_tf_idfs in tfidf.doc_term_tf_idfs.items():
    manual_size+=sys.getsizeof(docid)
    for term, tf_idf in term_tf_idfs.items():
        manual_size+=sys.getsizeof(term)
        manual_size+=sys.getsizeof(tf_idf)
print("we calculated:\t\t", manual_size)

I calculated:		 47381406
python calculated:	 295008
we calculated:		 66514316


In [4]:
#source: https://code.activestate.com/recipes/577504/
from __future__ import print_function
from sys import getsizeof, stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o))
            #print(s, type(o), repr(o), file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)

print("some link coming from python doc of sys.getsizeof() calculated",total_size(tfidf.doc_term_tf_idfs, verbose=False))

some link coming from python doc of sys.getsizeof() calculated 87970668


In [5]:
#source:
#Wissam Jarjoui, Measure the Real Size of Any Python Object, 21.07.2016, URL:https://goshippo.com/blog/measure-real-size-any-python-object/
def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

print("Wissam Jarjoui calculated", get_size(tfidf.doc_term_tf_idfs))

Wissam Jarjoui calculated 87970668
